## I solved the homework problems using Python, specifically the Pulp, Pandas and Numpy libraries.

# Question 1

[35 points] During each 4-hour period, the Smalltown police force requires the following
number of on-duty police officers: 8 officers from 12 midnight to 4am; 7 officers from 4am
to 8am; 6 officers from 8am to 12 noon; 6 officers from 12 noon to 4 pm; 5 officers from 4
to 8pm and 4 officers from 8pm to 12 midnight. Each officer works two consecutive 4-hour
shifts. Formulate an ILP that can be used to minimize the number of police officers needed
to meet Smalltown’s daily requirements. Use either EXCEL or GAMS to solve the
problem, and print out your EXCEL sheet or MPL codes and solutions.
(Hint: define 𝑥𝑡 as the number of police officers for shift 𝑡 (they start duty from time point
(𝑡 − 1) × 4); 𝑡 = 1, . . . , 6.)

### Variables

a_var = number of police officers available to work at 12am

b_var = number of police officers available to work at 4am

c_var = number of police officers available to work at 8am

d_var = number of police officers available to work at 12pm

e_var = number of police officers available to work at 4pm

f_var = number of police officers available to work at 8pm

### Constraints

a_var + b_var >= 8

b_var + c_var >= 7

c_var + d_var >= 6

d_var + e_var >= 6

e_var + f_var >= 5

f_var + a_var >= 4

### Solution

In [32]:
import pulp as pl

# declare variables
a_var = pl.LpVariable("a_var", 0, None, pl.LpInteger)
b_var = pl.LpVariable("b_var", 0, None, pl.LpInteger)
c_var = pl.LpVariable("c_var", 0, None, pl.LpInteger)
d_var = pl.LpVariable("d_var", 0, None, pl.LpInteger)
e_var = pl.LpVariable("e_var", 0, None, pl.LpInteger)
f_var = pl.LpVariable("f_var", 0, None, pl.LpInteger)

# define the problem
prob = pl.LpProblem("Smalltown_Police", pl.LpMinimize)

# objective function
prob += a_var + b_var + c_var + d_var + e_var + f_var

# constraints
prob += a_var + b_var >= 8
prob += b_var + c_var >= 7
prob += c_var + d_var >= 6
prob += d_var + e_var >= 6
prob += e_var + f_var >= 5
prob += f_var + a_var >= 4

status = prob.solve()  # solve
print(pl.LpStatus[status])  # print status

# print variable values
print("a_var", pl.value(a_var))
print("b_var", pl.value(b_var))
print("c_var", pl.value(c_var))
print("d_var", pl.value(d_var))
print("e_var", pl.value(e_var))
print("f_var", pl.value(f_var))

# print optimized objective function value
print("\nThe optimized objective function value is: ")
print(pl.value(prob.objective))
print('The minimum number of police officers needed is 19.')

Optimal
a_var 4
b_var 4
c_var 3
d_var 3
e_var 5
f_var 0

The optimized objective function value is: 
19
The minimum number of police officers needed is 19.


# Question 2

[35 points] Jobco Shop has 10 outstanding jobs to be processed on a single machine. The
following table provides processing times and due dates. All times are in days and due time
is measured from time 0:

In [33]:
problemstatement = pd.DataFrame(np.array([[1,10,20], [2,3,98], [3,13,100],  [4,15,34],
                           [5,9,50], [6,22,44], [7,17,32], 
                           [8,30,60], [9,12,80], [10,16,150]]),
                   columns=['Job', 'Processing_Time', 'Due_Time'])

problemstatement

,Job,Processing_Time,Due_Time
0,1,10,20
1,2,3,98
2,3,13,100
3,4,15,34
4,5,9,50
5,6,22,44
6,7,17,32
7,8,30,60
8,9,12,80
9,10,16,150


If job 4 precedes job 3, then job 9 must precede job 3. The objective is to process all 10
jobs in the shortest possible time. Formulate the model as an ILP. Use MPL to solve the
problem, and print out your MPL codes and solution. Use either EXCEL or GAMS to solve
the problem, and print out your EXCEL sheet or MPL codes and solutions.

In [31]:
### RUN THIS BLOCK ###

import pandas as pd
import numpy as np

df = pd.DataFrame(np.array([[1,10,20,10], [2,3,98,95], [3,13,100,87],  [4,15,34,19],
                           [5,9,50,41], [6,22,44,22], [7,17,32,15], 
                           [8,30,60,30], [9,12,80,68], [10,16,150,134]]),
                   columns=['Job', 'Processing_Time', 'Due_Time', 'Slack_Time'])

counter = 0
counterinside = 0
threshold = 180
delays = []

userinput = input('How many tables would you like to generate? \n(A good number is 10000. It\'ll take around a minute to load. The more you generate, the more accurate the result.) ')

#iterate x times (where x is the argument inside the range function)
for i in range(int(userinput)):

    # randomization
    dfr = df.sample(frac=1).reset_index(drop=True)

    # create Out_Time column
    dfr['Out_Time'] = dfr['Processing_Time'].cumsum()

    # create Delay_Time column
    dfr['Delay_Time'] = dfr['Out_Time'] - dfr['Due_Time']

    # setting all Delay_Time negative values to zero
    dfr.loc[dfr['Delay_Time'] < 0, 'Delay_Time'] = 0
    
    # add up all the values in the Delay_Time column
    columnsum = dfr['Delay_Time'].sum()
    
    counter = counter + 1
    
    if columnsum < threshold:
        threshold = columnsum
        dffinal = dfr
        counterinside = counterinside + 1
        
    else:
        pass
    
job_list = dffinal["Job"].tolist()

print(dffinal)
print('\nShown above is the dataframe with the least cumulative "Delay Time" out of the ' + userinput + ' created.')
print('\nThe best job-ordering configuration found: ')
print(job_list)
print('\nLowest Delay Time found: ')
print(sum(dffinal.Delay_Time))

print('\nThe dataframe was improved ' + str(counterinside) + ' times out of the ' + str(counter) + ' passes.')

How many tables would you like to generate? 
(A good number is 10000. It'll take around a minute to load. The more you generate, the more accurate the result.) 10000
   Job  Processing_Time  Due_Time  Slack_Time  Out_Time  Delay_Time
0    7               17        32          15        17           0
1    4               15        34          19        32           0
2    1               10        20          10        42          22
3    5                9        50          41        51           1
4    2                3        98          95        54           0
5    6               22        44          22        76          32
6    3               13       100          87        89           0
7    9               12        80          68       101          21
8    8               30        60          30       131          71
9   10               16       150         134       147           0

Shown above is the dataframe with the least cumulative "Delay Time" out of the 10000 

## My Methodology for Question 2

The program I wrote creates a predetermined amount of tables with randomly placed rows i.e. random job ordering. For each table, it calculates the out time and the delay time, and then sums up the total delay time.

In a kind of machine-learny way (but not really), the best table is selected from all the random tables. This table is hopefully close to the best possible job ordering.

After analysis of hundreds of thousands of randomly-ordered tables, the lowest total delay time that I found was 120. 

### The job order that achieves a 120 delay time is: 1-4-7-5-6-2-9-3-8-10


This is significantly lower than the standard "Closest-Due-Time-First" approach which yields a delay time of 180.

Of course, it is possible that there exists a delay time smaller than 120 from a table that my program did not generate. If a computer ran the code to generate millions and millions of random tables, then there would be a better chance that a table with the true smallest delay time would be generated. My laptop is way too old to do that, but that fact is  definitely worth noting.

# Question 3

[30 points] Jaco owns a plant in which three products are manufactured. The labor and raw
material requirements for the three products are given in the following table.
Product Required daily labor
(hr/unit)
Required daily raw
material (lb/unit)
1 3 4
2 4 3
3 5 6
Daily
Availability
100 100
The profits per unit for the three products are $25, $30, and $45, respectively. If product 3
is to be manufactured at all, then its production level must be at least 5 units daily.
Formulate the problem as a mixed integer linear program. Use either EXCEL or GAMS to
solve the problem, and print out your EXCEL sheet or MPL codes and solutions.

## 3a. When Product 3 is manufactured
### Variables

product_1 = number of product 1s manufactured

product_2 = number of product 2s manufactured

product_3 = number of product 3s manufactured


### Constraints

3 * product_1 + 4 * product_2 + 5 * product_3 <= 100

4 * product_1 + 3 * product_2 + 6 * product_3 <= 100

1 * product_3 >= 5 (because if product 3 is to be manufactured at all, then its production level must be at least 5 units daily)

In [22]:
import pulp as pl

# declare variables
product_1 = pl.LpVariable("product_1", 0, None, pl.LpInteger)
product_2 = pl.LpVariable("product_2", 0, None, pl.LpInteger)
product_3 = pl.LpVariable("product_3", 0, None, pl.LpInteger)

# define the problem
prob = pl.LpProblem("ManufacturingPlant", pl.LpMaximize)

# objective function
prob += 25 * product_1 + 30 * product_2 + 45 * product_3

# constraints
prob += 3 * product_1 + 4 * product_2 + 5 * product_3 <= 100
prob += 4 * product_1 + 3 * product_2 + 6 * product_3 <= 100
prob += 1 * product_3 >= 5

status = prob.solve()  # solve
print(pl.LpStatus[status])  # print status

# print variable values
print("product_1", pl.value(product_1))
print("product_2", pl.value(product_2))
print("product_3", pl.value(product_3))

# print optimized objective function value
print("\nThe optimized objective function value when product 3 is made is: ")
print(pl.value(prob.objective))

Optimal
product_1 0
product_2 11
product_3 11

The optimized objective function value when product 3 is made is: 
825


## 3b. When Product 3 is NOT manufactured
### Variables

product_1 = number of product 1s manufactured

product_2 = number of product 2s manufactured

product_3 = number of product 3s manufactured


### Constraints

3 * product_1 + 4 * product_2 + 5 * product_3 <= 100

4 * product_1 + 3 * product_2 + 6 * product_3 <= 100

1 * product_3 <= 0 (because if product 3 is to be manufactured at all, then its production level must be at least 5 units daily. It is not being manufactured.)

In [23]:
import pulp as pl

# declare variables
product_1 = pl.LpVariable("product_1", 0, None, pl.LpInteger)
product_2 = pl.LpVariable("product_2", 0, None, pl.LpInteger)
product_3 = pl.LpVariable("product_3", 0, None, pl.LpInteger)

# define the problem
prob = pl.LpProblem("ManufacturingPlant", pl.LpMaximize)

# objective function
prob += 25 * product_1 + 30 * product_2 + 45 * product_3

# constraints
prob += 3 * product_1 + 4 * product_2 + 5 * product_3 <= 100
prob += 4 * product_1 + 3 * product_2 + 6 * product_3 <= 100
prob += 1 * product_3 <= 0

status = prob.solve()  # solve
print(pl.LpStatus[status])  # print status

# print variable values
print("product_1", pl.value(product_1))
print("product_2", pl.value(product_2))
print("product_3", pl.value(product_3))

# print optimized objective function value
print("\nThe optimized objective function value when product 3 is not made is: ")
print(pl.value(prob.objective))

Optimal
product_1 12
product_2 16
product_3 0

The optimized objective function value when product 3 is not made is: 
780
